In [21]:
import sqlite3
import pandas as pd
# Import the recommendation functions
from rec import get_recommendations, cal_scores

db_path = 'hot100.db'

table_name = 'hot100'

with sqlite3.connect(db_path) as conn:
    df = pd.read_sql(f"SELECT * FROM {table_name};", conn)

In [22]:
cal_scores(df)

In [23]:
# 1. Get general recommendations
top_recommendations = get_recommendations(df, n=10)
print("Top 10 Recommended Songs:")
for rec in top_recommendations:
    print(f"{rec['rank']}. {rec['artist']} - {rec['song']} (Score: {rec['recommendation_score']:.4f})")

# 2. Get recommendations excluding songs already played
new_recommendations = get_recommendations(df, n=5, exclude_played=True)
print("\nTop 5 New Songs to Try:")
for rec in new_recommendations:
    print(f"{rec['id']} {rec['rank']}. {rec['artist']} - {rec['song']}")

# 3. Get recommendations by artist
kendrick_recs = get_recommendations_by_artist(df, "Kendrick", n=3)
print("\nTop Kendrick Lamar Songs Recommended:")
print(kendrick_recs)

# 4. Get recommendations by genre/tag
pop_recs = get_recommendations_by_tag(df, "pop", n=3)
print("\nTop Pop Songs Recommended:")
print(pop_recs)

Top 10 Recommended Songs:
1. Kendrick Lamar - Not Like Us (Score: 0.9336)
2. Kendrick Lamar Featuring Lefty Gunplay - TV Off (Score: 0.9209)
3. Kendrick Lamar - Squabble Up (Score: 0.5825)
4. ROSE & Bruno Mars - APT. (Score: 0.4366)
5. Lady Gaga & Bruno Mars - Die With A Smile (Score: 0.3421)
6. Billie Eilish - Birds Of A Feather (Score: 0.2136)
7. Sabrina Carpenter - Espresso (Score: 0.1508)
8. Drake - Nokia (Score: 0.1170)
9. Shaboozey - A Bar Song (Tipsy) (Score: 0.0857)
10. Post Malone Featuring Morgan Wallen - I Had Some Help (Score: 0.0768)

Top 5 New Songs to Try:
d1f54212-1f19-450c-8479-a299b05ef991 1. Benson Boone - Beautiful Things
c3a50de3-d5ef-4576-900b-e1b228b11d45 2. Morgan Wallen - Love Somebody
9e0a60b7-5a1e-482b-977e-11fe9530e305 3. Chappell Roan - Pink Pony Club
bcc31c63-ca0e-4b4b-bdf5-e2160af4fe46 4. The Weeknd & Playboi Carti - Timeless
f2b892bf-899c-4a0f-8029-4023c62a2ad9 5. Lola Young - Messy

Top Kendrick Lamar Songs Recommended:
                                 